In [1]:
import os
import glob
from tqdm.notebook import tqdm
import networkx as nx
import time
import pandas as pd

In [2]:
os.chdir(r'C:\Users\Kislee\PycharmProjects\Korean_textbook')
DATA_PATH = os.getcwd() + r'\target_data'
print(DATA_PATH)

C:\Users\Kislee\PycharmProjects\Korean_textbook\target_data


In [3]:
files = glob.glob(os.path.join(DATA_PATH, '*.xlsx'))

In [4]:
all_dfs = []
for source in files:
    temp_df = pd.read_excel(source, engine='openpyxl')
    all_dfs.append(temp_df)
df = pd.concat(all_dfs)

In [5]:
df.head()

학교명 등급                                                 지문  외국인인명   국가명  민족명
0  경희대  5  장민: 어제 요리 프로그램에서 제철 음식을 가지고 누가 더 맛 있게 만드는지 대결을...  장민/한나   NaN  NaN
1  경희대  5  흐엉: 장민, 이 제육볶음 좀 먹어 봐. 어제 마을 장터에 갔는데 직접 만든 고추장...     장민   NaN  NaN
2  경희대  5  은영: 승진 시험 준비는 잘 되어 가?\n상진: 그냥 그래. 요즘 일도 공부도 손에...     상진   NaN  NaN
3  경희대  5  의학 전문가: 여러분은 어떻게 휴식을 하십니까? 집에서 텔레비전을 보면서 쉬신다고요...    윤태훈   NaN  NaN
4  경희대  5  예술 교육은 우리의 삶에 어떤 영향을 줄까요? 그에 대한 하나의 대답으로 콜롬비아의...    NaN  콜롬비아  NaN

In [6]:
cell_df = df['지문']

In [7]:
#UTagger
from Utagger.bin.utagger_py import UTagger

In [8]:
os.chdir(r'C:\Users\Kislee\PycharmProjects\Korean_textbook\Utagger\한국어교재지문')
print(os.getcwd())

C:\Users\Kislee\PycharmProjects\Korean_textbook\Utagger\한국어교재지문


In [9]:
dllpath = r'../bin/UTaggerR64.dll'
cfgpath = r'../Hlxcfg.txt'

rt = UTagger.Load_global(dllpath, cfgpath)
ut = UTagger(0) # 0은 객체 고유번호. 0~99 지정 가능. 같은 번호로 여러번 생성하면 안됨. 한 스레드당 하나씩 지정 필요.
rt = ut.new_ucma() #객체 생성. 객체가 있어야 유태거 이용 가능.

python call utagger function
C:\Users\Kislee\PycharmProjects\Korean_textbook\Utagger\Hlxcfg.txt


In [10]:
cell_utagging = [ut.tag_line(cell,3) for cell in tqdm(cell_df)]

  0%|          | 0/475 [00:00<?, ?it/s]

In [11]:
ut.release_ucma() #객체 해제
UTagger.Release_global() #사전 해제

In [12]:
#단어 어깨번호 정제하고자 할 때 사용
def cutSemanticNum(word, all=False): # all : True이면 의미숫자를 다 잘라버림, False이면 두글자만 남김
    splitted = word.split('__')
    
    if len(splitted) == 1: # __ 구분자가 없는단어는 원 단어 그대로 반환
        return word
    
    if all==True:
        return splitted[0]
    
    out_word = splitted[0]+'__'+splitted[1][-2:]
    return out_word    

In [13]:
# utagger 가 생성한 한 어절에서 + 단위로 형태소를 자른 후 / 단위로 어휘와 태그를 분석
def utagging_to_simple_tagging(utg):
    #outlier가 발견되어 제거함
    utg = utg.replace('NNG\n','NNG')
    
    temp_wordset = utg.split('+') # utagger 가 생성한 결과에서 + 단위로 잘라줌
   
    wordset = []
    for word_tag in temp_wordset:
        seplist = word_tag.split('/') #/ 로 잘라지지 않는 경우에 대한 예외처리
        if len(seplist) <= 1 :
            continue
        else :
            wordset.append(seplist)
    if len(wordset) <= 0:
        return None
   
    try:
        first_word = wordset[0][0]
        first_tag = wordset[0][1]
    except:
        print("예외처리 : ", wordset)
    
    #동사(VV), 형용사(VA)로 시작하면 바로 첫 단어만 내보냄
    if first_tag.startswith('VV') or first_tag.startswith('VA'):
        return (first_word, first_tag)
    
    if len(wordset) == 1: #한 단어인 경우 
         #명사류, 부사, 한글자 이상의 외국어는 사용
        if first_tag.startswith('NNP') or first_tag.startswith('NNG') or first_tag.startswith('MAG') or (first_tag.startswith('SL') and len(first_word)>1) :
            #if word_tag[1].startswith('SL'):
            #            print(word_tag[0])
            return (first_word, first_tag)

    else: #여러 단어인 경우 
        if first_tag.startswith('NNP') or first_tag.startswith('NNG') or first_tag.startswith('SL') or first_tag.startswith('XPN'): #첫 단어가 다음과 같으면
            output = []
            
            if first_tag.startswith('XPN'): #XPN+NNG 는 합쳐서 NNG로
                second_word = wordset[1][0]
                second_tag = wordset[1][1]
                if second_tag.startswith('NNG'):
                        output.append((cutSemanticNum(first_word,True)+second_word, second_tag))
                        #print(output[0])
                return output
            
            for word_tag in wordset: #XPN+NNG 로 반환해야함--> NNG로
                if word_tag[1].startswith('NNP') or word_tag[1].startswith('NNG') or (word_tag[1].startswith('SL') and len(word_tag[0])>1): #명사류를 찾음
                    #if word_tag[1].startswith('SL'): #지울 것
                    #    print(word_tag[0])
                    output.append((word_tag[0], word_tag[1]))
            return output

    return None

#문장 전체를 받아서 (단어,태그) 튜플 리스트로 반환
def remain_available_words(doc): 
    result = []
    for utg in doc.split(' '):
        simple_tagged = utagging_to_simple_tagging(utg)
        if simple_tagged == None:
            continue
                
        if str(type(simple_tagged)) =="<class 'tuple'>":
            result.append(simple_tagged)
        else :
            for word_tag in simple_tagged:
                result.append(word_tag)
    return result

In [14]:
cell_tagging = [remain_available_words(cell) for cell in cell_utagging]

In [15]:
#불용어 수집
os.chdir(r'C:\Users\Kislee\PycharmProjects\Korean_textbook')
DATA_PATH = os.getcwd()
filename = DATA_PATH+r'\불용어목록.xlsx'
stopword_df = pd.read_excel(filename, engine='openpyxl')

In [16]:
#불용어 제거 필터
stopword = list(stopword_df['불용어'])
def stopword_filter(cell):
    output = []
    for word, tag in cell:
        if word not in stopword:
            output.append((word,tag))
    return output      

In [17]:
cell_tagging = [stopword_filter(cell) for cell in cell_tagging]

In [18]:
#태그:빈도 사전구성
tag_freq_dic = {}
for cell in cell_tagging:
    for word, tag in cell:
        tag_freq_dic[tag] = tag_freq_dic.get(tag, 0) + 1
print("고유 태그 수 : %d" % len(tag_freq_dic))

고유 태그 수 : 6


In [19]:
#단어:태그 사전구성
word_tag_dic = {}
for cell in cell_tagging:
    for word, tag in cell:
        word_tag_dic[word] = tag

In [20]:
#셀내의 튜플 리스트에서 워드만 추출
def word_selector(cell):
    output = []
    for word, tag in cell:
        if word not in stopword:
            output.append(word)
    return output      

In [21]:
cell_wordlist = [word_selector(cell) for cell in cell_tagging]

In [22]:
#EDA
print("지문 수 : ", len(cell_wordlist))

지문 수 :  475


In [23]:
cnt = 0
for cell in cell_wordlist:
    cnt += len(cell)
print('단어 출현 수 :', cnt)

단어 출현 수 : 91307


In [24]:
#단어:빈도 사전
word_freq_dic = {}
for cell in cell_wordlist:
    for word in cell:
        word_freq_dic[word] = word_freq_dic.get(word, 0) + 1
print("고유 단어 수 : %d" % len(word_freq_dic))

고유 단어 수 : 11272


In [25]:
word_freq = []
for key, value in word_freq_dic.items():
    word_freq.append((value, key))
word_freq.sort(reverse=True)
print("상위단어 정렬", word_freq[:1000])

상위단어 정렬 [(2890, '있__01'), (1038, '보__01'), (897, '사람'), (624, '같'), (539, '없__01'), (463, '많'), (437, '남자__02'), (433, '생각하'), (430, '좋__01'), (428, '여자__02'), (415, '한국__05'), (338, '일__01'), (338, '더__01'), (329, '말__01'), (322, '만들'), (297, '문제__06'), (293, '크__01'), (285, '잘__02'), (284, '가__01'), (278, '알'), (271, '듣__01'), (268, '사회__07'), (266, '시간__04'), (262, '받__01'), (254, '학생'), (252, '사회자'), (249, '오늘'), (241, '생각__01'), (236, '살__01'), (235, '들__01'), (231, '많이'), (231, '가지'), (223, '지금__03'), (214, '자신__01'), (212, '먹__02'), (209, '주__01'), (200, '또'), (199, '오__01'), (193, '경우__03'), (189, '문화__01'), (188, '중요하__02'), (188, '쓰__03'), (187, '아이__01'), (184, '따르__01'), (179, '좀__02'), (179, '안__02'), (173, '말하'), (173, '느끼__02'), (171, '필요하'), (171, '경제__04'), (170, '영화__01'), (167, '안녕하'), (164, '가장__01'), (163, '나오'), (162, '맞__01'), (162, '교수__06'), (158, '세계__02'), (157, '이야기'), (156, '음악__01'), (153, '사용하__03'), (151, '사실__04'), (150, '말씀'), (150, '다양하__01'), (147, '

In [26]:
# 단어 붙이기
cell_wordjoin = [' '.join(cutSemanticNum(word) for word in cell) for cell in cell_wordlist]

In [27]:
#파일 저장을 위한 일회성 코드
#out_df = pd.DataFrame(cell_utagging)
#out_df.to_excel('utag_result1012.xlsx')

In [28]:
#국가 목록
country_list = ['중국', '미국', '일본', '나우루', '영국', '호주', '스페인', '독일', '인도', '그리스', '러시아', '프랑스', '노르웨이', '콜롬비아', '이탈리아', '브라질', '몽골', '태국', '이란', '몰디브', '모나코', '필리핀', '파라과이', '투발루', '코스타리카', '케냐', '캐나다', '자메이카', '이집트', '예멘', '에티오피아', '스웨덴', '부탄', '베트남', '라오스', '파나마', '탄자니아', '칠레', '인도네시아', '스위스', '스리랑카', '뉴질랜드', '나이지리아', '과테말라']

In [29]:
#입력 리스트에 대한 쌍 조합 튜플을 반환
def get_pairs_from_list(lst):
    pairs = []
    unique = list(set(lst)) #리스트 내 중복 제거
    lst_len = len(unique)
    for i in range(lst_len):
        if i==(lst_len-1):
            break
        subset = unique[i+1:]

        for item in subset:
            pairs.append((unique[i], item))
    return pairs

In [30]:
#국가명으로 연관단어 네트워크 분석
def country_semantic_network(country):
    #대상 셀 선정
    target_cell_wordjoin = [cell for cell in cell_wordjoin if country in cell]
    #단어 합치기
    target_cell_wordlist = [cell.split(" ") for cell in target_cell_wordjoin]
    word_dic = {}
    for cell in target_cell_wordlist:
        for word in cell:
            word_dic[word] = word_dic.get(word, 0) + 1
    print("총 단어 수 : %d" % len(word_dic))
    word_freq = []
    for key, value in word_dic.items():
        word_freq.append((value, key))
    word_freq.sort(reverse=True)
    #네트워크 생성, gephi를 위한 source-target 생성
    edges = [get_pairs_from_list(cell) for cell in target_cell_wordlist]
    G = nx.Graph()
    for comb in edges:
        for source, dest in comb:
            if G.has_edge(source, dest):
                G[source][dest]['weight'] += 1
            else:
                G.add_edge(source, dest, weight=1)
    print("생성된 그래프 노드 수: %d, 엣지 수 : %d" %(G.number_of_nodes(), G.number_of_edges()))
    ####degree centrality 추출 및 소팅(시간 소요됨)
    start = time.time()
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    print("네트워크 분석 완료 time : %.2f seconds" % (time.time()-start))
    #####degree 추출 및 소팅
    degree = G.degree()
    degree_freq = []
    for key, value in degree:
        degree_freq.append((value, key))
    degree_freq.sort(reverse=True)
    #print("상위 degree", degree_freq[:100])
    
    #결과 데이터 생성
    country_df = pd.DataFrame()
    country_df = pd.DataFrame(columns=['품사','단어','출현빈도','출현빈도/품사빈도','연결단어수','연결중심성','매개중심성','분석범위'])
    for dfreq, word in degree_freq[:200] :
        country_df = country_df.append({'품사':word_tag_dic[word],'단어':word,'출현빈도':word_dic[word],'연결단어수':dfreq,'연결중심성':degree_centrality[word],'매개중심성':betweenness_centrality[word],'분석범위':country,'출현빈도/품사빈도':word_freq_dic[word]/tag_freq_dic[word_tag_dic[word]]}, ignore_index=True)

    DATA_PATH = os.getcwd() + r'\save_data'
    filename = DATA_PATH+'\지문단위국가_'+country+'.xlsx'
    country_df.to_excel(filename)
    print("%s 저장 완료!" %filename)


In [31]:
for country in country_list:
    print("================== %s 분석 시작 =====================" % country)
    country_semantic_network(country)
    print("================== %s 분석 종료 =====================" % country)
    print()

================== 중국 분석 시작 =====================
총 단어 수 : 2608
생성된 그래프 노드 수: 2608, 엣지 수 : 454423
네트워크 분석 완료 time : 338.47 seconds
C:\Users\Kislee\PycharmProjects\Korean_textbook\save_data\지문단위국가_중국.xlsx 저장 완료!
================== 중국 분석 종료 =====================

================== 미국 분석 시작 =====================
총 단어 수 : 2712
생성된 그래프 노드 수: 2712, 엣지 수 : 455336
네트워크 분석 완료 time : 347.71 seconds
C:\Users\Kislee\PycharmProjects\Korean_textbook\save_data\지문단위국가_미국.xlsx 저장 완료!
================== 미국 분석 종료 =====================

================== 일본 분석 시작 =====================
총 단어 수 : 1924
생성된 그래프 노드 수: 1924, 엣지 수 : 283001
네트워크 분석 완료 time : 144.87 seconds
C:\Users\Kislee\PycharmProjects\Korean_textbook\save_data\지문단위국가_일본.xlsx 저장 완료!
================== 일본 분석 종료 =====================

================== 나우루 분석 시작 =====================
총 단어 수 : 423
생성된 그래프 노드 수: 423, 엣지 수 : 89253
네트워크 분석 완료 time : 6.60 seconds
C:\Users\Kislee\PycharmProjects\Korean_textbook\save_data\지문단위국가_나우루.xlsx 저장 완료!
======

C:\Users\Kislee\PycharmProjects\Korean_textbook\save_data\지문단위국가_스웨덴.xlsx 저장 완료!
================== 스웨덴 분석 종료 =====================

================== 부탄 분석 시작 =====================
총 단어 수 : 265
생성된 그래프 노드 수: 265, 엣지 수 : 20385
네트워크 분석 완료 time : 1.24 seconds
C:\Users\Kislee\PycharmProjects\Korean_textbook\save_data\지문단위국가_부탄.xlsx 저장 완료!
================== 부탄 분석 종료 =====================

================== 베트남 분석 시작 =====================
총 단어 수 : 189
생성된 그래프 노드 수: 189, 엣지 수 : 9510
네트워크 분석 완료 time : 0.41 seconds
C:\Users\Kislee\PycharmProjects\Korean_textbook\save_data\지문단위국가_베트남.xlsx 저장 완료!
================== 베트남 분석 종료 =====================

================== 라오스 분석 시작 =====================
총 단어 수 : 379
생성된 그래프 노드 수: 379, 엣지 수 : 71631
네트워크 분석 완료 time : 4.73 seconds
C:\Users\Kislee\PycharmProjects\Korean_textbook\save_data\지문단위국가_라오스.xlsx 저장 완료!
================== 라오스 분석 종료 =====================

================== 파나마 분석 시작 =====================
총 단어 수 : 160
생성된 그래프 노드 수: 160, 엣지 수 : 1

In [32]:
#코드 종료